In [ ]:
group_data = {}
for label, data in ean._connection_data.items():
    if data["source_component"] not in group_data:
        group_data[data["source_component"]] = pd.DataFrame(columns=["chemical", "physical", "massless"])

    group_data[data["source_component"]].loc[data["target_component"]] = [data.get("E_CH", 0), data.get("E_PH", 0), data.get("E", 0)]

for connlabel in ean.E_F_dict["inputs"]:
    group_data["E_F"].loc[ean._connection_data[connlabel]["target_component"]] = [0, 0, 0]

for component_name, component in ean.components.items():
    if not hasattr(component, "E_D"):
        E_D = 0
    else:
        E_D = component.E_D
    group_data[component_name].loc["E_D"] = [0, 0, E_D]